# Additive Backend

> The additive backend can be used to obtain pathlengths and group delays.

In [ ]:
from functools import partial

import sax

## Parametrized Models

In [ ]:
def coupler(length=50.0) -> sax.SDict:
    sdict = {
        ("in0", "out0"): length,
        ("in0", "out1"): length,
        ("in1", "out0"): length,
        ("in1", "out1"): length,
    }
    return sax.reciprocal(sdict)

In [ ]:
def waveguide(length=100.0) -> sax.SDict:
    sdict = {
        ("in0", "out0"): length,
    }
    return sax.reciprocal(sdict)

## Circuit with additive backend

In [ ]:
mzi, _ = sax.circuit(
    netlist={
        "instances": {
            "lft": coupler,
            "top": partial(waveguide, length=500),
            "btm": partial(waveguide, length=100),
            "rgt": coupler,
        },
        "connections": {
            "lft,out0": "btm,in0",
            "btm,out0": "rgt,in0",
            "lft,out1": "top,in0",
            "top,out0": "rgt,in1",
        },
        "ports": {
            "in0": "lft,in0",
            "in1": "lft,in1",
            "out0": "rgt,out0",
            "out1": "rgt,out1",
        },
    },
    backend="additive",
)

In [ ]:
mzi()